In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from scipy.stats import ttest_1samp

In [57]:
df=pd.read_csv("bitcoin.csv")
df.head()

,Date,Open,High,Low,Close,Volume,Month,Day,Year,Daily_Return,7-day MA,30-day MA,30-day Volatility
0,2015-01-01,320.434998,320.434998,314.002991,314.248993,8036550,1,1,2015,NaN,NaN,NaN,NaN
1,2015-01-02,314.079010,315.838989,313.565002,315.032013,7860650,1,2,2015,0.002492,NaN,NaN,NaN
2,2015-01-03,314.846008,315.149994,281.082001,281.082001,33054400,1,3,2015,-0.107767,NaN,NaN,NaN
3,2015-01-04,281.145996,287.230011,257.612000,264.195007,55629100,1,4,2015,-0.060079,NaN,NaN,NaN
4,2015-01-05,265.084015,278.341003,265.084015,274.473999,43962800,1,5,2015,0.038907,NaN,NaN,NaN


In [58]:
df.shape

(3560, 13)

In [59]:
df=df.dropna()

In [60]:
df.shape

(3530, 13)

In [61]:
df.tail()

,Date,Open,High,Low,Close,Volume,Month,Day,Year,Daily_Return,7-day MA,30-day MA,30-day Volatility
3555,2024-09-25,64302.589844,64804.503906,62945.375000,63143.144531,25078377700,9,25,2024,-0.018022,63421.699777,59364.688411,0.023105
3556,2024-09-26,63138.546875,65790.796875,62669.269531,65181.019531,36873129847,9,26,2024,0.032274,63741.780134,59553.917969,0.021436
3557,2024-09-27,65180.664062,66480.695312,64852.992188,65790.664062,32058813449,9,27,2024,0.009353,64112.878348,59779.352604,0.021355
3558,2024-09-28,65792.179688,66255.531250,65458.035156,65887.648438,15243637984,9,28,2024,0.001474,64468.993862,59996.001563,0.021355
3559,2024-09-29,65888.898438,66069.343750,65450.015625,65635.304688,14788214575,9,29,2024,-0.003830,64752.792969,60213.195833,0.021346


In [62]:
X=df.drop(['Close','Date'],axis=1)
y=df['Close']

In [63]:
X.head(2)

,Open,High,Low,Volume,Month,Day,Year,Daily_Return,7-day MA,30-day MA,30-day Volatility
30,226.440994,233.503998,216.309006,23348200,1,31,2015,-0.039576,243.140429,246.600333,0.074958
31,216.867004,231.574005,212.014999,29128500,2,1,2015,0.043722,239.319571,243.664999,0.075558


In [64]:
df.corr(numeric_only=True)

,Open,High,Low,Close,Volume,Month,Day,Year,Daily_Return,7-day MA,30-day MA,30-day Volatility
Open,1.000000,0.999538,0.999243,0.998959,0.628968,-0.047854,0.002329,0.813488,-0.033024,0.998858,0.991136,-0.016372
High,0.999538,1.000000,0.999104,0.999556,0.634823,-0.049355,0.002364,0.811854,-0.016545,0.998658,0.990859,-0.011006
Low,0.999243,0.999104,1.000000,0.999446,0.618180,-0.045635,0.002848,0.816171,-0.014835,0.998102,0.990252,-0.023512
Close,0.998959,0.999556,0.999446,1.000000,0.627586,-0.048053,0.002179,0.813715,-0.001515,0.998133,0.990135,-0.016286
Volume,0.628968,0.634823,0.618180,0.627586,1.000000,-0.098359,-0.003855,0.619723,-0.016775,0.629837,0.617450,0.155395
Month,-0.047854,-0.049355,-0.045635,-0.048053,-0.098359,1.000000,0.009664,-0.078726,0.006828,-0.047959,-0.046878,-0.158286
Day,0.002329,0.002364,0.002848,0.002179,-0.003855,0.009664,1.000000,-0.001388,-0.003474,0.003324,0.006341,-0.007675
Year,0.813488,0.811854,0.816171,0.813715,0.619723,-0.078726,-0.001388,1.000000,-0.019494,0.814140,0.815397,-0.093690
Daily_Return,-0.033024,-0.016545,-0.014835,-0.001515,-0.016775,0.006828,-0.003474,-0.019494,1.000000,-0.028582,-0.033242,0.011857
7-day MA,0.998858,0.998658,0.998102,0.998133,0.629837,-0.047959,0.003324,0.814140,-0.028582,1.000000,0.993837,-0.016093


In [65]:
X.head(2)

,Open,High,Low,Volume,Month,Day,Year,Daily_Return,7-day MA,30-day MA,30-day Volatility
30,226.440994,233.503998,216.309006,23348200,1,31,2015,-0.039576,243.140429,246.600333,0.074958
31,216.867004,231.574005,212.014999,29128500,2,1,2015,0.043722,239.319571,243.664999,0.075558


In [66]:
y

30        217.464005
31        226.972000
32        238.229004
33        227.268005
34        226.852997
            ...     
3555    63143.144531
3556    65181.019531
3557    65790.664062
3558    65887.648438
3559    65635.304688
Name: Close, Length: 3530, dtype: float64

In [67]:
# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_noisy, test_size=0.25, random_state=0)

In [68]:
# Shuffle the target variable y_train
y_shuffled = y_train.sample(frac=1, random_state=42).reset_index(drop=True)

# Train the model with a Random Forest and adjust hyperparameters
model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_shuffled)

# Predict using the model on the test data
y_pred = model.predict(X_test)

# Evaluate using R² score
print("Shuffled y R²:", r2_score(y_test, y_pred))

Shuffled y R²: -0.034081599304172316


In [ ]:
# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# Fit on training data and transform both training and test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
from sklearn.ensemble import RandomForestRegressor
  
 # create regressor object
model = RandomForestRegressor(n_estimators = 20, random_state = 0)
  
# fit the regressor with x and y data
model.fit(X_train, y_train) 
y_pred = model.predict(X_test)

In [12]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('r2_score:',r2_score(y_test,y_pred))

Mean Absolute Error: 178.73222080718895
Mean Squared Error: 154717.46644660443
Root Mean Squared Error: 393.34141206667323
r2_score: 0.9996338875247284


In [13]:
# Metrics calculation
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(y_test, y_pred)))

# Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('Mean Absolute Percentage Error (MAPE):', mape)

# R-squared score
print('R-squared (R²):', r2_score(y_test, y_pred))

Mean Absolute Error (MAE): 178.73222080718895
Mean Squared Error (MSE): 154717.46644660443
Root Mean Squared Error (RMSE): 393.34141206667323
Mean Absolute Percentage Error (MAPE): 1.0739878849475357
R-squared (R²): 0.9996338875247284


In [33]:
import scipy.stats as stats

# Hypothesis Test
# Null Hypothesis (H0): The mean difference between predicted and actual values is 0 (no difference).
# Alternative Hypothesis (H1): The mean difference between predicted and actual values is not 0 (significant difference).

# Calculate the difference between predicted and actual values
difference = y_test - y_pred.flatten()

# Perform a paired t-test
t_stat, p_value = stats.ttest_1samp(difference, 0)

# Display the t-statistic and p-value
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Conclusion of Hypothesis Test
if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant difference between predicted and actual values.")
else:
    print("Fail to reject the null hypothesis: No significant difference between predicted and actual values.")

T-statistic: -0.922805073955601
P-value: 0.3563611612789864
Fail to reject the null hypothesis: No significant difference between predicted and actual values.


In [34]:

rf_model = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=-1)

cv = KFold(n_splits=5, shuffle=True, random_state=0)

mae_scores_train = -cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')
mse_scores_train = -cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
r2_scores_train = cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='r2')

# Fit model on full training data and evaluate on test set
rf_model.fit(X_train, y_train)
y_pred_test = rf_model.predict(X_test)

# Calculate Testing Performance
mae_test = mean_absolute_error(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred_test)

# Print Training Performance (CV Mean ± Std)
print(f"Training Performance (5-Fold CV):")
print(f"MAE: {mae_scores_train.mean():.2f} ± {mae_scores_train.std():.2f}")
print(f"MSE: {mse_scores_train.mean():.2f} ± {mse_scores_train.std():.2f}")
print(f"RMSE: {np.sqrt(mse_scores_train).mean():.2f} ± {np.sqrt(mse_scores_train).std():.2f}")
print(f"R²: {r2_scores_train.mean():.4f} ± {r2_scores_train.std():.4f}")

# Print Testing Performance
print("\nTesting Performance:")
print(f"MAE: {mae_test:.2f}")
print(f"MSE: {mse_test:.2f}")
print(f"RMSE: {rmse_test:.2f}")
print(f"R²: {r2_test:.4f}")




Training Performance (5-Fold CV):
MAE: 190.38 ± 11.27
MSE: 145113.26 ± 17646.06
RMSE: 380.22 ± 23.41
R²: 0.9996 ± 0.0001

Testing Performance:
MAE: 172.61
MSE: 141695.30
RMSE: 376.42
R²: 0.9997


In [44]:
from sklearn.ensemble import RandomForestRegressor

# Initialize Random Forest with pruning
rf_model = RandomForestRegressor(
    n_estimators=100, 
    max_depth=10,                 # Limit tree depth
    min_samples_split=5,          # Prevent splitting small nodes
    min_samples_leaf=2,           # Prevent very small leaf nodes
    random_state=0, 
    n_jobs=-1
)

# Fit and evaluate as before

cv = KFold(n_splits=5, shuffle=True, random_state=0)

mae_scores_train = -cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='neg_mean_absolute_error')
mse_scores_train = -cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
r2_scores_train = cross_val_score(rf_model, X_train, y_train, cv=cv, scoring='r2')

# Fit model on full training data and evaluate on test set
rf_model.fit(X_train, y_train)
y_pred_test = rf_model.predict(X_test)

# Calculate Testing Performance
mae_test = mean_absolute_error(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred_test)

# Print Training Performance (CV Mean ± Std)
print(f"Training Performance (5-Fold CV):")
print(f"MAE: {mae_scores_train.mean():.2f} ± {mae_scores_train.std():.2f}")
print(f"MSE: {mse_scores_train.mean():.2f} ± {mse_scores_train.std():.2f}")
print(f"RMSE: {np.sqrt(mse_scores_train).mean():.2f} ± {np.sqrt(mse_scores_train).std():.2f}")
print(f"R²: {r2_scores_train.mean():.4f} ± {r2_scores_train.std():.4f}")

# Print Testing Performance
print("\nTesting Performance:")
print(f"MAE: {mae_test:.2f}")
print(f"MSE: {mse_test:.2f}")
print(f"RMSE: {rmse_test:.2f}")
print(f"R²: {r2_test:.4f}")


Training Performance (5-Fold CV):
MAE: 191.71 ± 11.44
MSE: 148848.66 ± 19033.01
RMSE: 385.00 ± 25.01
R²: 0.9996 ± 0.0001

Testing Performance:
MAE: 174.42
MSE: 146816.09
RMSE: 383.17
R²: 0.9997


In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4]  
}

# Grid search with cross-validation
grid_search = GridSearchCV(RandomForestRegressor(random_state=0, n_jobs=-1), 
                           param_grid, 
                           cv=5, 
                           scoring='neg_mean_absolute_error', 
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

# Print best parameters
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


#### overfiting if using this hyperparameter tuning proposed parameter